In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch
from keras_tuner.tuners import Hyperband
from tensorflow import keras

import pickle

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X.astype("float32")/255.0

class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        
        model = Sequential()
        
        # Convolutional 1
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_1',
                    values=[16, 32, 64],
                    default=32,
                ),
                activation='relu',
                kernel_size=3
            )
        )

        # Pooling 1
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # Dropout 1
        model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.0,
                max_value=0.3,
                default=0.1,
                step=0.05,
            ))
        )
        
        # Convolutional 2
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_2',
                    values=[16, 32, 64],
                    default=32,
                ),
                activation='relu',
                kernel_size=3
            )
        )

        # Pooling 2
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # Dropout 2
        model.add(
            Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.0,
                max_value=0.3,
                default=0.1,
                step=0.05,
            ))
        )
        
        # Convolutional 3
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_3',
                    values=[16, 32, 64],
                    default=32,
                ),
                activation='relu',
                kernel_size=3
            )
        )

        # Pooling 3
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        # Dropout 3
        model.add(
            Dropout(rate=hp.Float(
                'dropout_3',
                min_value=0.0,
                max_value=0.3,
                default=0.1,
                step=0.05,
            ))
        )
        
        # Flatten
        model.add(Flatten())
        
        # Hidden layer 1
        model.add(
            Dense(
                units=hp.Int(
                    'units_1',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_1',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        
        # Dropout 4
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_4',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        
        # Hidden layer 2
        model.add(
            Dense(
                units=hp.Int(
                    'units_2',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation_2',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        
        # Dropout 5
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_5',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )

        model.add(Dense(self.num_classes, activation='sigmoid'))

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=0.0008),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        return model
    
NUM_CLASSES = 1  # happy sad number of classes
INPUT_SHAPE = (48, 48, 1)  # happy sad images input shape
SEED = 1
HYPERBAND_MAX_EPOCHS = 15
MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='random_search',
    project_name='HappySad_7'
)

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    seed=SEED,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='hyperband',
    project_name='HappySad_7'
)

N_EPOCH_SEARCH = 15
BATCH_SIZE = 16

tuner.search(X, y, batch_size=BATCH_SIZE, epochs=N_EPOCH_SEARCH, validation_split=0.1)

tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

Trial 30 Complete [00h 08m 39s]
val_accuracy: 0.8281573355197906

Best val_accuracy So Far: 0.8498964905738831
Total elapsed time: 01h 23m 51s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in hyperband\HappySad_7
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
Cov_filters_1: 32
dropout_1: 0.1
Cov_filters_2: 64
dropout_2: 0.2
Cov_filters_3: 16
dropout_3: 0.25
units_1: 192
dense_activation_1: relu
dropout_4: 0.35000000000000003
units_2: 256
dense_activation_2: tanh
dropout_5: 0.45
tuner/epochs: 15
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8498964905738831
Trial summary
Hyperparameters:
Cov_filters_1: 32
dropout_1: 0.2
Cov_filters_2: 64
dropout_2: 0.05
Cov_filters_3: 64
dropout_3: 0.15000000000000002
units_1: 32
dense_activation_1: relu
dropout_4: 0.30000000000000004
units_2: 288
dense_activation_2: sigmoid
dropout_5: 0.15000000000000002
tuner/epochs: 15
tuner/initial_epoch: 5
tuner/bracket: 2
tune

In [2]:
# Output best result
tuner.results_summary()

pickle_out = open("best_model.pickle", "wb")
pickle.dump(tuner, pickle_out)
pickle_out.close()

loss, accuracy = best_model.evaluate(X, y)

Results summary
Results in hyperband\HappySad_7
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
Cov_filters_1: 32
dropout_1: 0.1
Cov_filters_2: 64
dropout_2: 0.2
Cov_filters_3: 16
dropout_3: 0.25
units_1: 192
dense_activation_1: relu
dropout_4: 0.35000000000000003
units_2: 256
dense_activation_2: tanh
dropout_5: 0.45
tuner/epochs: 15
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8498964905738831
Trial summary
Hyperparameters:
Cov_filters_1: 32
dropout_1: 0.2
Cov_filters_2: 64
dropout_2: 0.05
Cov_filters_3: 64
dropout_3: 0.15000000000000002
units_1: 32
dense_activation_1: relu
dropout_4: 0.30000000000000004
units_2: 288
dense_activation_2: sigmoid
dropout_5: 0.15000000000000002
tuner/epochs: 15
tuner/initial_epoch: 5
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 14a2486ef0dbdbda00268009aa1fb1d8
Score: 0.846273273229599
Trial summary
Hyperparameters:
Cov_filters_1: 32
dropout_1: 0.0
Cov_filters_2: 64
dropout_2: 0.1